# Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import zipfile

# Load Data

In [2]:
full_price = pd.read_csv("/Users/cindy/quandl_data_table_downloads/QUOTEMEDIA/QUOTEMEDIA_PRICES_20230614.csv")
full_price.head()

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,JTKWY,2022-03-11,6.1700,7.3200,5.790,6.72,9440097.0,0.0,1.0,6.1700,7.3200,5.790,6.72,9440097.0
1,JTKWY,2022-03-10,6.1600,6.1750,5.935,6.07,2261623.0,0.0,1.0,6.1600,6.1750,5.935,6.07,2261623.0
2,FG_1,2020-06-01,8.1000,8.3900,8.100,8.39,3086317.0,0.0,1.0,8.1000,8.3900,8.100,8.39,3086317.0
3,FLWS,2022-03-09,14.5700,14.9588,14.410,14.45,662492.0,0.0,1.0,14.5700,14.9588,14.410,14.45,662492.0
4,RENW_,2020-01-29,21.9768,21.9900,21.970,21.99,319.0,0.0,1.0,21.9768,21.9900,21.970,21.99,319.0


In [3]:
# S&P 500 tickers list
SP500 = pd.read_excel("S&P 500 tickers.xlsx")
SP500.head()

,Company,Symbol,Weight
0,Apple Inc.,AAPL,6.634317
1,Microsoft Corporation,MSFT,5.627977
2,Amazon.com Inc.,AMZN,2.508250
3,Alphabet Inc. Class A,GOOGL,1.638209
4,Berkshire Hathaway Inc. Class B,BRK.B,1.629718


In [4]:
# Fama-French 5 factors returns
F_F_5_Factors = pd.read_csv("F_F_Research_Data_5_Factors_daily.csv", skiprows = 3)
F_F_5_Factors.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
F_F_5_Factors['Date'] = pd.to_datetime(F_F_5_Factors['Date'], format='%Y%m%d')
F_F_5_Factors.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012


# Universe
- S&P 500 stocks that are updated recently
- At least 5 years of price data (2018-01-01 to 2023-05-31), IPO after 2018-01-01 are excluded
- Have at least 100 trading days of price data before 2018-01-01 for rolling window Fama-French exposure calculation (IPO before 2017-08-16)
- Because of the lacking data of Fama-French 5 factors returns, the universe is updated to 2018-01-01 to 2023-04-28

In [5]:
# Only keep the stocks that are in the S&P 500
universe = full_price[full_price['ticker'].isin(SP500['Symbol'])].copy().sort_values(by = ['ticker', 'date'])
universe['date'] = pd.to_datetime(universe['date'])
universe

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
34609545,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.783619,32.729252,26.183402,28.801742,44739900.0
8276274,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,28.107882,28.147157,26.059030,26.432144,10897100.0
8276273,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,27.040908,28.801742,26.222677,28.801742,4705200.0
8276272,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,27.819864,28.559545,26.347048,26.347048,4274400.0
8276271,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,26.268498,27.453297,26.183402,26.877262,3464400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5882272,ZTS,2023-06-07,170.14,170.14,162.84,163.58,3195076.0,0.0,1.0,170.140000,170.140000,162.840000,163.580000,3195076.0
6750729,ZTS,2023-06-08,162.96,164.87,162.38,164.30,1983997.0,0.0,1.0,162.960000,164.870000,162.380000,164.300000,1983997.0
45051669,ZTS,2023-06-09,163.73,164.23,162.95,163.09,1346569.0,0.0,1.0,163.730000,164.230000,162.950000,163.090000,1346569.0
17458601,ZTS,2023-06-12,163.84,165.61,162.76,165.55,1422996.0,0.0,1.0,163.840000,165.610000,162.760000,165.550000,1422996.0


In [6]:
# Check the date of 100 trading days prior to 2018-01-01
universe['date'].tolist()[universe['date'].tolist().index(dt.datetime(2018, 1, 9)) - 100]

Timestamp('2017-08-16 00:00:00')

If we filter out the stocks that are IPO after 2017-08-15, we will exclude 15 tickers from our universe.

In [7]:
# List of the stocks that are IPO after 2017-08-16
SP500_start_date = universe.groupby('ticker').date.min().to_frame().reset_index().rename(columns = {'date': 'start_date'})
SP500_start_date[SP500_start_date.start_date >= '2017-08-16']

,ticker,start_date
28,AMCR,2019-06-11
75,CARR,2020-04-03
82,CDAY,2018-04-26
86,CEG,2022-02-02
120,CTVA,2019-06-03
126,DD,2017-09-01
138,DOW,2019-03-20
196,GEHC,2023-01-04
229,HWM,2020-02-24
272,LHX,2019-07-01


In [8]:
universe.drop(universe[(universe.ticker.isin(SP500_start_date[SP500_start_date.start_date > '2017-08-16'].ticker))].index, inplace = True)

In [9]:
# Set up the universe time series precisely
universe = universe[(universe['date'] >= '2017-08-15') & (universe['date'] < '2023-06-01')].sort_values(by = ['ticker', 'date'])[['ticker', 'date', 'adj_close', 'adj_volume']].reset_index(drop = True)
# Calculate the daily return for each ticker
universe['return'] = universe.groupby('ticker')['adj_close'].pct_change()
universe = universe.dropna()
universe.head()

,ticker,date,adj_close,adj_volume,return
1,A,2017-08-16,59.499989,6891973.0,0.046480
2,A,2017-08-17,58.274370,3501640.0,-0.020599
3,A,2017-08-18,58.647801,2357872.0,0.006408
4,A,2017-08-21,59.116983,1486144.0,0.008000
5,A,2017-08-22,60.285152,1666369.0,0.019760


# Fama-French 5 factors

- The Fama-French 5 factors returns are updated daily.
- The Fama-French 5 factors returns are risk premiums of the market which are not directly comparable to the returns of the stocks.
- We will run the regression of daily excess returns of each stock on the Fama-French 5 factors to get the beta values as that stock's exposures to each risk premium in the model.
- To prevent look ahead bias, I will use the expected return of the previous 100 days as the exposure to each risk premium of th next day. In other words, we will run 100 trading days rolling window regression of daily excess returns against Fama-French 5 factors returns.
- Since the most recent factor return data that is available is 2023-04-28, our universe is updated to 2018-01-01 to 2023-04-28.

In [10]:
# Match the date of Fama-French 5 factors to the universe date time series
F_F_5_Factors = F_F_5_Factors[(F_F_5_Factors['Date'] >= '2017-08-15') & (F_F_5_Factors['Date'] < '2023-06-01')].reset_index(drop = True)
F_F_5_Factors

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2017-08-15,-0.11,-0.87,-0.02,-0.17,-0.25,0.004
1,2017-08-16,0.18,-0.14,-0.44,0.33,-0.14,0.004
2,2017-08-17,-1.60,-0.18,-0.17,-0.04,0.14,0.004
3,2017-08-18,-0.15,0.11,0.24,-0.27,-0.01,0.004
4,2017-08-21,0.06,-0.27,-0.22,0.19,-0.14,0.004
...,...,...,...,...,...,...,...
1431,2023-04-24,0.00,-0.27,0.47,0.66,0.28,0.018
1432,2023-04-25,-1.76,-0.83,0.10,0.44,0.89,0.018
1433,2023-04-26,-0.41,-0.04,-0.75,0.05,-0.33,0.018
1434,2023-04-27,1.85,-0.59,0.00,0.46,-0.19,0.018


In [11]:
F_F_regression = universe.merge(F_F_5_Factors, left_on = 'date', right_on = 'Date', how = 'left')
F_F_regression['excess_return'] = F_F_regression['return'] - F_F_regression['RF']
# The time range of the universe shrinks because of the lacking information of Fama-French 5 factor returns data
F_F_regression = F_F_regression.dropna()
F_F_regression

,ticker,date,adj_close,adj_volume,return,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_return
0,A,2017-08-16,59.499989,6891973.0,0.046480,2017-08-16,0.18,-0.14,-0.44,0.33,-0.14,0.004,0.042480
1,A,2017-08-17,58.274370,3501640.0,-0.020599,2017-08-17,-1.60,-0.18,-0.17,-0.04,0.14,0.004,-0.024599
2,A,2017-08-18,58.647801,2357872.0,0.006408,2017-08-18,-0.15,0.11,0.24,-0.27,-0.01,0.004,0.002408
3,A,2017-08-21,59.116983,1486144.0,0.008000,2017-08-21,0.06,-0.27,-0.22,0.19,-0.14,0.004,0.004000
4,A,2017-08-22,60.285152,1666369.0,0.019760,2017-08-22,1.06,-0.02,-0.24,-0.17,0.18,0.004,0.015760
...,...,...,...,...,...,...,...,...,...,...,...,...,...
706757,ZTS,2023-04-24,176.350000,1117741.0,-0.002996,2023-04-24,0.00,-0.27,0.47,0.66,0.28,0.018,-0.020996
706758,ZTS,2023-04-25,173.020000,1151148.0,-0.018883,2023-04-25,-1.76,-0.83,0.10,0.44,0.89,0.018,-0.036883
706759,ZTS,2023-04-26,172.940000,1205198.0,-0.000462,2023-04-26,-0.41,-0.04,-0.75,0.05,-0.33,0.018,-0.018462
706760,ZTS,2023-04-27,173.950000,1682893.0,0.005840,2023-04-27,1.85,-0.59,0.00,0.46,-0.19,0.018,-0.012160


In [12]:
# Run 100 trading days rolling window regression of daily excess returns against Fama-French 5 factors returns
F_F_regression['Mkt-RF_beta'] = np.nan
F_F_regression['SMB_beta'] = np.nan
F_F_regression['HML_beta'] = np.nan
F_F_regression['RMW_beta'] = np.nan
F_F_regression['CMA_beta'] = np.nan
for ticker, group in F_F_regression.groupby('ticker'):
    for idx in range(100, len(group)):
        X = group.loc[group.index[idx] - 100:group.index[idx] - 1, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
        y = group.loc[group.index[idx] - 100:group.index[idx] - 1, 'excess_return']
        #print(group.loc[group.index[idx]])
        coefficients = np.linalg.inv(X.T @ X) @ X.T @ y
        F_F_regression.loc[group.index[idx], 'Mkt-RF_beta'] = coefficients[0]
        F_F_regression.loc[group.index[idx], 'SMB_beta'] = coefficients[1]
        F_F_regression.loc[group.index[idx], 'HML_beta'] = coefficients[2]
        F_F_regression.loc[group.index[idx], 'RMW_beta'] = coefficients[3]
        F_F_regression.loc[group.index[idx], 'CMA_beta'] = coefficients[4]
        #print(group.loc[group.index[idx]])

In [13]:
# Drop the unnecessary columns which are market risk premiums
F_F_regression.drop(columns=['Date','Mkt-RF','SMB','HML','RMW','CMA','RF','excess_return'], inplace=True)

In [189]:
# Drop the first 100 days' missing values of beta for each ticker
# The universe moves back to '2018-01-01' to '2023-04-28'
F_F_regression = F_F_regression.dropna()
F_F_regression.to_pickle('stock_universe.pkl')

In [14]:
F_F_regression.dropna()

,ticker,date,adj_close,adj_volume,return,Mkt-RF_beta,SMB_beta,HML_beta,RMW_beta,CMA_beta
100,A,2018-01-09,69.013902,2666711.0,0.024554,0.009027,0.001063,-0.007793,-0.004662,-0.005455
101,A,2018-01-10,68.071536,2957184.0,-0.013655,0.008722,0.000859,-0.006795,-0.006242,-0.005700
102,A,2018-01-11,68.081152,1511134.0,0.000141,0.007732,0.001027,-0.007521,-0.005757,-0.005344
103,A,2018-01-12,68.975438,1448155.0,0.013136,0.007377,0.000628,-0.007505,-0.005661,-0.005886
104,A,2018-01-16,68.494639,1703398.0,-0.006971,0.007580,0.000491,-0.007457,-0.005694,-0.005725
...,...,...,...,...,...,...,...,...,...,...
706757,ZTS,2023-04-24,176.350000,1117741.0,-0.002996,0.008080,0.004188,0.000523,-0.001081,-0.001064
706758,ZTS,2023-04-25,173.020000,1151148.0,-0.018883,0.007815,0.004184,0.000368,-0.001543,-0.000980
706759,ZTS,2023-04-26,172.940000,1205198.0,-0.000462,0.007942,0.004660,0.000837,-0.001425,-0.001464
706760,ZTS,2023-04-27,173.950000,1682893.0,0.005840,0.008008,0.004670,0.001035,-0.001673,-0.001201
